In [1]:
import pandas as pd
import requests
import time
import json
import re
import os

In [2]:
group_domain = 'spb_today'

In [3]:
service_token = '<put_your_service_token_here>'
access_token = service_token

## Fetch data from VK

In [21]:
def vk_get_wall_posts(**params):
    url = 'https://api.vk.com/method/wall.get'
    try:
        res = requests.get(url, params={
            **params,
            'v': '5.124',
            'access_token': access_token,
        })
        if not res.ok:
            return None
        if 'error' in res:
            print(res.json())
            return None
        return res.json()
    except:
        return None

In [28]:
res = vk_get_wall_posts(domain=group_domain, offset=5, count=1)
res['response']['items'][0]

{'id': 13979013,
 'from_id': -68471405,
 'owner_id': -68471405,
 'date': 1603222583,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': 'Паровозик из 4х машин на Пискаревском над Шафировской развязкой.',
 'signer_id': 35464517,
 'attachments': [{'type': 'video',
   'video': {'access_key': '8ba0aaebd56e84e8a1',
    'can_comment': 0,
    'can_like': 1,
    'can_repost': 1,
    'can_subscribe': 1,
    'can_add_to_faves': 1,
    'can_add': 1,
    'comments': 0,
    'date': 1603222371,
    'description': '',
    'duration': 17,
    'image': [{'height': 96,
      'url': 'https://sun9-45.userapi.com/AitiE7CwqH-MA4vHBcR0dHsNeuH9auB4sO0Pmg/UUMd0XqV7u8.jpg',
      'width': 130,
      'with_padding': 1},
     {'height': 120,
      'url': 'https://sun9-70.userapi.com/Msu1i7igGbAzer1M6VIVz0_Aabf6hKu3pritqQ/aJ7UXnWguFM.jpg',
      'width': 160,
      'with_padding': 1},
     {'height': 240,
      'url': 'https://sun9-56.userapi.com/K9tgRlYsB1mDcclQOE-Ws_A-n2mj6wAAd7_Wng/aaK7QPYGFKQ.jpg',
      'wid

In [29]:
def get_posts(domain: str):
    offset = 0
    count = 100
    while True:
        if offset % 1000 == 0:
            print('offset', offset)
        data = vk_get_wall_posts(domain=domain, offset=offset, count=count)
        if not data:
            return None
        items = data['response']['items']
        if len(items) == 0:
            return None
        for item in items:
            yield item
        
        offset += count
        time.sleep(0.2)

In [ ]:
posts = list(get_posts(group_domain))

In [20]:
json.dump(posts, open('20201020-vk-spb_today.json', 'w'), ensure_ascii=False)

## Create DataFrame

In [78]:
def create_row(post):
    return {
        'id': post['id'],
        'text': post['text'],
        'post_source': post['post_source']['type'],
        'comments': post['comments']['count'],
        'likes': post['likes']['count'],
        'reposts': post['reposts']['count'],
        'views': post.get('views', {}).get('count', 0),
        'from_id': post['from_id'],
        'owner_id': post['owner_id'],
        'signer_id': post.get('signer_id'),
        'date': post['date'],
        'ads': post['marked_as_ads'],
#         'post_type': post['post_type'],
        'attachments_count': len(post.get('attachments', [])),
    }

In [71]:
df = pd.DataFrame([create_row(x) for x in posts])
df.set_index('id', inplace=True)
df.head(5)

,text,post_source,comments,likes,reposts,views,from_id,owner_id,signer_id,date,ads,post_type,attachments_count
id,,,,,,,,,,,,,
13979023,Новости нашего мегаполиса:\n\n1. Дефицит бюдже...,vk,34,19,0,20049,-68471405,-68471405,NaN,1603222707,0,post,9
13979092,В ночь с 19 на 20 октября был угнан Рено Дасте...,vk,17,10,0,22263,-68471405,-68471405,22394006.0,1603223824,0,post,3
13979085,Сегодня в промежутке 17.40-17.55 на парковке у...,vk,59,9,0,22659,-68471405,-68471405,22648.0,1603223753,0,post,1
13979013,Паровозик из 4х машин на Пискаревском над Шафи...,vk,8,15,0,23230,-68471405,-68471405,35464517.0,1603222583,0,post,1
13978872,Мазда и Шкода устроили дтп на Гренадерской ули...,vk,6,23,2,33686,-68471405,-68471405,425196.0,1603220496,0,post,2


In [86]:
# drop ads posts
df = df[df['ads'] == 0] 

In [87]:
df = df[df['text'].notnull()]

In [88]:
df = df[['text', 'date', 'comments', 'likes', 'reposts', 'views', 'attachments_count', 'from_id', 'owner_id', 'signer_id']]
df.head(5)

,text,date,comments,likes,reposts,views,attachments_count,from_id,owner_id,signer_id
id,,,,,,,,,,
13979023,Новости нашего мегаполиса:\n\n1. Дефицит бюдже...,1603222707,34,19,0,20049,9,-68471405,-68471405,NaN
13979092,В ночь с 19 на 20 октября был угнан Рено Дасте...,1603223824,17,10,0,22263,3,-68471405,-68471405,22394006.0
13979085,Сегодня в промежутке 17.40-17.55 на парковке у...,1603223753,59,9,0,22659,1,-68471405,-68471405,22648.0
13979013,Паровозик из 4х машин на Пискаревском над Шафи...,1603222583,8,15,0,23230,1,-68471405,-68471405,35464517.0
13978872,Мазда и Шкода устроили дтп на Гренадерской ули...,1603220496,6,23,2,33686,2,-68471405,-68471405,425196.0


In [89]:
df.to_csv('20201020-vk-spb_today.csv')